<a href="https://colab.research.google.com/github/xivh/stylegan2-colab-notebooks/blob/master/Bucket_ckpt_to_Drive_pkl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Convert a ckpt in GCS to a pkl and save it in Google Drive

Original code written by https://github.com/shawwn

# Set up StyleGAN2 and authenticate

In [ ]:
!git clone https://github.com/shawwn/stylegan2 -b estimator /content/stylegan2
%cd /content/stylegan2
!git checkout working

In [ ]:
# magical solution: https://stackoverflow.com/a/59492999/9919772
!gcloud auth application-default login

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Get StyleGAN2 ready for your network

Set label size to something other than  0 if you have labels

Set resolution to your network resolution

In [ ]:
# --- set resolution and label size here:
label_size = 0    #@param {type:"integer"}
resolution = 512  #@param {type:"integer"}
# ------------------------

Make sure your runtime type is TPU or this part won't work

You may get an error about the TPU not having access permissions for your bucket, you can find info in the docs at https://cloud.google.com/tpu/docs/storage-buckets or by pasting the error into Google - some people say you should deauthorize the TPU once you're finished since this is just a random TPU from Colab

In [ ]:
%tensorflow_version 1.x
%cd /content/stylegan2

import os

os.environ['TPU_NAME'] = 'grpc://' + os.environ['COLAB_TPU_ADDR']
os.environ['NOISY'] = '1'

import tqdm
from pprint import pprint as pp
from training.networks_stylegan2 import *

import dnnlib
from dnnlib import EasyDict

import tensorflow as tf
import tflex
import os

dnnlib.tflib.init_tf()

sess = tf.get_default_session()
sess.list_devices()

cores = tflex.get_cores()
tflex.set_override_cores(cores)

#synthesis_func          = 'G_synthesis_stylegan2'
#kwargs = {'resolution': 512}
#synthesis = tflib.Network('G_synthesis', func_name=globals()[synthesis_func], **kwargs)

#sess.reset(os.environ['TPU_NAME']) # don't do this, this breaks the session

train     = EasyDict(run_func_name='training.training_loop.training_loop') # Options for training loop.
G_args    = EasyDict(func_name='training.networks_stylegan2.G_main')       # Options for generator network.
D_args    = EasyDict(func_name='training.networks_stylegan2.D_stylegan2')  # Options for discriminator network.
G_opt     = EasyDict(beta1=0.0, beta2=0.99, epsilon=1e-8)                  # Options for generator optimizer.
D_opt     = EasyDict(beta1=0.0, beta2=0.99, epsilon=1e-8)                  # Options for discriminator optimizer.
G_loss    = EasyDict(func_name='training.loss.G_logistic_ns_pathreg')      # Options for generator loss.
D_loss    = EasyDict(func_name='training.loss.D_logistic_r1')              # Options for discriminator loss.
sched     = EasyDict()                                                     # Options for TrainingSchedule.
grid      = EasyDict(size='8k', layout='random')                           # Options for setup_snapshot_image_grid().
sc        = dnnlib.SubmitConfig()                                          # Options for dnnlib.submit_run().
tf_config = {'rnd.np_random_seed': 1000}                                   # Options for tflib.init_tf().
num_channels = 3
label_dtype = np.int64
sched.minibatch_gpu = 1

if 'G' not in globals():
  with tflex.device('/gpu:0'):
    G = tflib.Network('G', num_channels=num_channels, resolution=resolution, label_size=label_size, **G_args)
    G.print_layers()
    Gs, Gs_finalize = G.clone2('Gs')
    Gs_finalize()
    D = tflib.Network('D', num_channels=num_channels, resolution=resolution, label_size=label_size, **D_args)
    D.print_layers()

def rand_latent(n, seed=None):
  if seed is not None:
    if seed < 0:
      seed = 2*32 - seed
    np.random.seed(seed)
  result = np.random.randn(n, *G.input_shape[1:])
  if seed is not None:
    np.random.seed()
  return result

def tfinit():
  tflib.run(tf.global_variables_initializer())

tfinit()

# Convert ckpt to pkl and save in Google Drive

Set the save directory, model directory, and checkpoints below

If you want to use the pkl with Colab/your own GPU, I recommend saving it to Google Drive to save on egress costs - change mv to gsutil mv if you want to put in back in your bucket instead

In [ ]:
save_to = '/content/drive/My Drive/networks' #@param {type:"string"}
model_dir = 'gs://your-bucket/runs/your-run'  #@param {type:"string"}

In [ ]:
#!gcloud auth login
from training import misc
G.print_layers();
D.print_layers();
saver = tf.train.Saver();

save_to = save_to.strip().rstrip('/') + '/' # rstrip for /content/...
model_dir = model_dir.strip().rstrip('/') + '/'
ckpt = tf.train.latest_checkpoint(model_dir)
print(ckpt)
assert ckpt is not None
saver.restore(sess, ckpt)
import os
step = int(os.path.basename(ckpt).split('-')[-1])
dst = os.path.basename(os.path.dirname(ckpt)) + '-{}.pkl'.format(step)
print('Saving to {}'.format(dst))
misc.save_pkl((G, D, Gs), dst)
print('Moving {} to {}{}'.format(dst, save_to, dst))
!mv *.pkl "$save_to" # comment this for gsutil
#!gsutil mv *.pkl $save_to # uncomment these for gsutil
#print('Making {}{} publicly readable...'.format(save_to, dst))
#!gsutil acl ch -u AllUsers:R $save_to$dst
#print('Model available at https://storage.googleapis.com/{}{}'.format(save_to.lstrip('gs://'), dst))